In [1]:
# Importing all need library
import os
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import librosa

Using TensorFlow backend.


In [2]:
# path of audio file
path = './audio/'

In [3]:
# Change into labels, index and one hot encoding
def get_label(PATH=path):
    lables = os.listdir(PATH)
    label_index = np.arange(0, len(lables))
    return lables, label_index, to_categorical(label_index)

In [4]:
# Vectorise the audio
def Wav2MFCC(path, max_len=11):
    wav, sr = librosa.load(path, mono=True, sr=None)
    wav = wav[::3]
    mfcc = librosa.feature.mfcc(wav, sr=16000)
    
    if(max_len>mfcc.shape[1]):
        pad_width = max_len-mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc    

In [5]:
# save data
def save_data_to_array(path=path, max_len=11):
    labels, _, _=get_label(path)
    for label in labels:
        mfcc_vector = []
        wavefile = [path+label+'/'+wavefile for wavefile in os.listdir(path + '/' + label)]
        
        for wave in wavefile:
            mfcc = Wav2MFCC(wave, max_len=max_len)
            mfcc_vector.append(mfcc)
        np.save(label+'.npy', mfcc_vector)

In [6]:
#spliting the data
def split_train_test():
    lables, index, _ = get_label(path)
    X = np.load(lables[0]+'.npy')
    y = np.zeros(X.shape[0])
    
    #Append all datasets into in single array
    for i, label in enumerate(lables[1:]):
        x = np.load(label+'.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value=(i+1)))
        
    assert X.shape[0]==len(y)
    
    return train_test_split(X, y, test_size=0.25, random_state=42)